In [ ]:
import json
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import date
import time

f = open('companieslist.json')
data = json.load(f)
f.close()
for (k, v) in data.items():
   driver = webdriver.Chrome()
   driver.get(str(v)) 
   length = driver.find_elements(By.XPATH, "//div[@data-bind='foreach: opportunities']/div")
   print("Total Number of Positions",len(length))
   time.sleep(4) 
   listofpositions = [] 
   for i in range(0,len(length)): 
      printlink = driver.find_element(By.XPATH, f"//div[@data-bind='foreach: opportunities']//div[@id='Opportunity{i}']//div[@class='row']//div//h3//a").get_attribute("href")
      name = driver.find_element(By.XPATH, f"//div[@data-bind='foreach: opportunities']//div[@id='Opportunity{i}']//div[@class='row']//div[2]//h3")
      if (name.text == "Oct 27, 2023"): 
         listofpositions.append(printlink) 
      else:
         continue
   driver.close()
   for i in range(len(listofpositions)):
      driver = webdriver.Chrome()
      driver.get(str(listofpositions[i]))  
      time.sleep(3)

# with open("SMTP.py") as f:
#    exec(f.read())
